In [ ]:
!pip install torch torchvision torchaudio
!pip install easyocr
!pip install gmm

In [ ]:
!pip uninstall opencv-python-headless==4.5.5.62 
!pip install opencv-python-headless==4.1.2.30

In [ ]:
import easyocr
#download the model
reader = easyocr.Reader(['tr','en'], gpu = False)

In [ ]:
# show an image
import PIL
import cv2
from PIL import ImageDraw
image_path='page3.jpg'
im = PIL.Image.open(image_path)
#im = PIL.Image.open("france.jfif")
width,height=im.size

print(width,height)

im

In [ ]:
bounds = reader.readtext(image_path)
bounds

In [ ]:
# Draw bounding boxes
def draw_boxes(image, bounds, color='red', width=2):
    draw = ImageDraw.Draw(image)
    for bound in bounds:
        p0, p1, p2, p3 = bound[0]
        draw.line([*p0, *p1, *p2, *p3, *p0], fill=color, width=width)
    return image

draw_boxes(im, bounds)

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
print(img.shape)
points=[]
points_2=[]
points_3 = []
for bound in bounds:
    p0, p1, p2, p3 = bound[0]
    midX=int(p0[0])
    midY=int((p0[1]+p1[1]+p2[1]+p3[1])/4)
    arr=(midX,midY)
    points.append(arr)
    x1=midX/width
    y1=midY/height
    arr2=(x1,y1)
    print(p0,p1,p2,p3)
    #points_2.append(arr2)
    points_2.append((p0[0]/width,p0[0]/width))
    points_2.append((p2[0]/width,p2[0]/width))
    points_3.append((p0[1]/height,p0[1]/height))
    points_3.append((p2[1]/height,p2[1]/height))
    cv2.circle(img,(midX,midY), 10, (0,0,255), -1)

cv2_imshow(img)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
X=np.array(points_2)#orta noktaları numpy array'e çevirdim.

In [ ]:
#x ler için GMM
gmm = GaussianMixture(n_components=8).fit(X)
labels = gmm.predict(X)
plt.gca().invert_yaxis()
plt.scatter(X[:, 0], np.zeros(len(X[:,0])), c=labels, s=60,  cmap='viridis');

X_df = pd.DataFrame(X)
gmm_cluster = pd.DataFrame(labels)  

df = pd.concat([X_df, gmm_cluster], axis=1)
df.columns = ['Feature 1', 'Feature 2', 'gmm_cluster']
df.head()

view_cluster = df['gmm_cluster'].value_counts().T
view_cluster = pd.DataFrame(data=view_cluster)
view_cluster = view_cluster.reset_index()
view_cluster.columns = ['gmm_cluster', 'count']
view_cluster.sort_values(by='gmm_cluster', ascending=False)


In [ ]:
#Centerları bul gmm x
cluster_count=len(gmm_cluster.value_counts())
centers=[]
for i in range(0,cluster_count-1):
  points_of_cluster= X[labels==i,:]
  centroid_of_cluster = np.mean(points_of_cluster, axis=0)
  centers.append(centroid_of_cluster)
  
arr_center=np.array(centers)
print(arr_center)
plt.vlines(arr_center,0,1,color='blue')
plt.grid()
plt.show()

In [ ]:
#yler için 
Y=np.array(points_3)#orta noktaları numpy array'e çevirdim.
gmm = GaussianMixture(n_components=8).fit(Y)
labels_y = gmm.predict(Y)
plt.gca().invert_yaxis()
plt.scatter(Y[:, 0], np.zeros(len(Y[:,0])), c=labels_y, s=60,  cmap='viridis');

Y_df = pd.DataFrame(Y)
gmm_cluster = pd.DataFrame(labels_y)  

df = pd.concat([Y_df, gmm_cluster], axis=1)
df.columns = ['Feature 1', 'Feature 2', 'gmm_cluster']
df.head()

view_cluster = df['gmm_cluster'].value_counts().T
view_cluster = pd.DataFrame(data=view_cluster)
view_cluster = view_cluster.reset_index()
view_cluster.columns = ['gmm_cluster', 'count']
view_cluster.sort_values(by='gmm_cluster', ascending=False)


In [ ]:
#Centerları bul gmm Y
cluster_count=len(gmm_cluster.value_counts())
centers_y=[]
for i in range(0,cluster_count-1):
  points_of_cluster= Y[labels==i,:]
  centroid_of_cluster = np.mean(points_of_cluster, axis=0)
  centers_y.append(centroid_of_cluster)
  
arr_center_y=np.array(centers_y)
print(arr_center_y)
plt.hlines(arr_center_y,0,1,color='red')
plt.grid()
plt.show()

In [ ]:
#Aynı resim üzerinde çizimi

plt.vlines(arr_center,0,1,color='blue')
plt.hlines(arr_center_y,0,1,color='red')
plt.grid()
plt.show()

In [ ]:
arr_lines_x=(arr_center*width)
arr_lines_y=(arr_center_y*height)
data_int_x = arr_lines_x.astype(int)
data_int_y = arr_lines_y.astype(int)

plt.vlines(data_int_x,2000,1,color='blue')
plt.hlines(data_int_y,1500,1,color='red')

plt.imshow(img)